# Cinemática Inversa

## Base de dados

In [1]:
import os
from pandas import read_csv
from numpy import array
class save:
    def __init__(self,path_project):
        self.path_project = path_project
        
    def dataframe(self,data,path):
        path_data = os.path.join(self.path_project,path)
        data.to_csv(path_data,index=False)

class extract:
    def __init__(self,path_project):
        self.path_project = path_project

    def dataframe(self,path):
        path_data = os.path.join(self.path_project,path)
        return read_csv(path_data)
    
def partition(dataset,axis):
    return array(dataset.iloc[:,axis[0]:axis[1]])

In [2]:
import sys

path_module = [r'C:\Users\je7560\Downloads\matheus\kinematics-robotics\src\kinematicsrobotics',
               r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\kinematicsrobotics']
sys.path.append(path_module[1])

path_data = r'src\data\ready\dataset-radius-0.5cm.csv'
path_project = [r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics',
                r'C:\Users\je7560\Downloads\matheus\kinematics-robotics']

ext = extract(path_project[0])

dataset = ext.dataframe(path_data)

y = partition(dataset,[0,4])
x = partition(dataset,[5,11])

### Análise da base de dados

In [3]:
print(f'Atributos e saídas:\n {dataset.columns}')
print(f"\nTamanho saída: {y.shape}\nTamanho entrada: {x.shape}\n")
print(f"Primeiras sáidas:\n{y[0:3]}\n")
print(f"Primeiras entrada:\n{x[0:3]}\n")


Atributos e saídas:
 Index(['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5', 'p_x', 'p_y',
       'p_z', 'roll', 'pich', 'yaw'],
      dtype='object')

Tamanho saída: (15324, 4)
Tamanho entrada: (15324, 6)

Primeiras sáidas:
[[ 0  0  0  0]
 [ 0  0 12  0]
 [ 0  0 12 12]]

Primeiras entrada:
[[ 5.40000000e+01 -1.10218212e-15  1.00000000e+01  2.35619449e+00
  -1.57079633e+00  7.85398163e-01]
 [ 5.32133136e+01 -2.47712141e-15  2.51517913e+00  3.14159265e+00
  -1.36135682e+00  2.75203954e-16]
 [ 5.36066568e+01 -2.24796486e-15  6.25758957e+00  2.33481267e+00
  -1.57079633e+00  8.06779988e-01]]



## Pré-processamento dos dados

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from numpy import mean,std

def size_split(size_train,size_val):
    size_val = (1 - size_train/(size_train+size_val))
    size_train = size_train/(size_train+size_val)
    return size_train,size_val


def split(x, y, test_size):
    return train_test_split(x, y, test_size=test_size)

def zscore(data):
    scaler = StandardScaler()
    means = mean(data,axis=0)
    stds = std(data,axis=0)
    data = scaler.fit_transform(data)
    return data, means,stds

In [5]:
size_train = 0.7
size_val = 0.2
size_test = 0.1

size_train,size_val = size_split(size_train,size_val)

x_train, x_test, y_train, y_test = split(x,y,test_size=size_test)


x_train,_,_ = zscore(x_train)
y_train,_,_ = zscore(y_train)

x_test,_,_ = zscore(x_test)
y_test,_,_ = zscore(y_test)

#### Análises

In [6]:
print(f"treino: {size_train}\nTeste: {size_test}\nValidação: {size_val}")
print(f"Tamanho dos dados de treino: {len(x_train)}")
print(f"Tamanho dos dados de teste: {len(x_test)}")

treino: 0.7590361445783133
Teste: 0.1
Validação: 0.2222222222222222
Tamanho dos dados de treino: 13791
Tamanho dos dados de teste: 1533


## Modelos e métricas

In [7]:
from sklearn.neural_network import MLPRegressor
def MLP():
    LEARNING_RATE = 0.01
    LEARNING = 'constant'
    EPOCHS = 2000
    EPOCHS_NOCHANGE = 10
    ERROR = 1e-4
    VERDOSE = False
    model = MLPRegressor(learning_rate_init = LEARNING_RATE,
                         learning_rate = LEARNING,
                         max_iter = EPOCHS,
                         tol = ERROR,
                         n_iter_no_change = EPOCHS_NOCHANGE,
                         early_stopping = True,
                         verbose=VERDOSE)
    return model


In [8]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def predict_mse(model,X_test,Y_test):
    y_predic = model.predict(X_test)
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')

def curve_loss(model):
    plt.plot(model.loss_curve_)
    plt.title('Erro Quadratico')
    plt.xlabel('Épocas')
    plt.ylabel('MSE')
    plt.show()

## Estimação dos hiperparâmetros

In [21]:
from sklearn.model_selection import ShuffleSplit,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from pandas import DataFrame


def space_hidden(min,max,incr,param_grid):
    hidden = [(i, j) for i in range(min, max,incr) for j in range(min, max, incr)]
    
    func_act =  ['relu', 'tanh']
    
    param_grid.update({
        'hidden_layer_sizes': hidden,
        'activation': func_act
    })

    return param_grid

def custom_mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)


def RandomizedSearch(x, y, model, param_grid, cv):
    random_search = RandomizedSearchCV(estimator=model, 
                                       param_distributions=param_grid, 
                                       scoring=custom_scorer, 
                                       cv=cv, 
                                       n_jobs=-1, 
                                       random_state=42, 
                                       return_train_score=True, 
                                       verbose=2)


def GridSearch(x, y, model, param_grid, cv):

    custom_scorer = make_scorer(custom_mse, greater_is_better=False)

    gridshearch = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               cv=cv,
                               scoring=custom_scorer,
                               return_train_score=True)
    
    gridshearch.fit(x, y)
    
    history = DataFrame(gridshearch.cv_results_)

    return history

def holdout(n_splits,test_size):
    return ShuffleSplit(n_splits=n_splits, 
                      test_size=test_size, 
                      random_state=42)


def grid_search_test(X_train, y_train, param_grid, model):


    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    #.ravel()

    # Avaliação do melhor hiperparâmetro
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)

    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print(f"\nMSE nos dados completos: {mse}")
    print(f"R² nos dados completos: {r2}")

    # Extrair os resultados da validação cruzada
    resultados = pd.DataFrame(grid_search.cv_results_)

    colunas_interesse = ['param_hidden_layer_sizes', 'mean_test_score', 'std_test_score']
    resultados_interesse = resultados[colunas_interesse]

    print("Avaliação do MSE para cada hiperparâmetro testado:")
    print(resultados_interesse)
    return resultados 


In [26]:
# Espaço de busca do grid search
min= 10
max = 50
incr = 10

param_grid= {}

param_grid = space_hidden(min,max,incr,param_grid)

# validação cruzada hold out
n_splits = 5

cv = holdout(n_splits,size_val)

# inicializando a rede
mlp = MLP()

history = GridSearch(x_train, y_train, mlp, param_grid, cv)



In [27]:
history.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_activation', 'param_hidden_layer_sizes', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'split3_train_score',
       'split4_train_score', 'mean_train_score', 'std_train_score'],
      dtype='object')

### Infomações

In [13]:
print(f"Número de parametros de camada oculta: {len(param_grid['hidden_layer_sizes'])}")

Número de parametros de camada oculta: 16


## Treinamento, validação e teste do modelo selecionado

In [60]:
mlp = MLP()
mlp.set_params(hidden_layer_sizes=(100, 200))

mlp.fit(x_train, y_train)

Iteration 1, loss = 0.08197672
Validation score: 0.928473
Iteration 2, loss = 0.02191331
Validation score: 0.968638
Iteration 3, loss = 0.01366824
Validation score: 0.973660
Iteration 4, loss = 0.01045488
Validation score: 0.978643
Iteration 5, loss = 0.00949138
Validation score: 0.980258
Iteration 6, loss = 0.00853995
Validation score: 0.980536
Iteration 7, loss = 0.00730065
Validation score: 0.979455
Iteration 8, loss = 0.00745579
Validation score: 0.982546
Iteration 9, loss = 0.00700912
Validation score: 0.980227
Iteration 10, loss = 0.00687598
Validation score: 0.984645
Iteration 11, loss = 0.00732972
Validation score: 0.968084
Iteration 12, loss = 0.00720072
Validation score: 0.982833
Iteration 13, loss = 0.00621128
Validation score: 0.936929
Iteration 14, loss = 0.00837502
Validation score: 0.988773
Iteration 15, loss = 0.00538541
Validation score: 0.956320
Iteration 16, loss = 0.00627883
Validation score: 0.978296
Iteration 17, loss = 0.00548673
Validation score: 0.988204
Iterat

MLPRegressor(early_stopping=True, hidden_layer_sizes=(100, 200),
             learning_rate_init=0.01, max_iter=2000, verbose=True)